## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [6]:
import numpy as np
import os
from random import shuffle
import re
!pip install bokeh


Processing /Users/sevenw/Library/Caches/pip/wheels/fb/f8/47/09700d9a19cbcbf0b7a3130690b75c0d6ff80fbda0b1774c7c/bokeh-1.4.0-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/98/42/87c585dd3b113c775e65fd6b8d9d0a43abe1819c471d7af702d4e01e9b20/packaging-20.1-py2.py3-none-any.whl
Processing /Users/sevenw/Library/Caches/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191/PyYAML-5.3-cp37-cp37m-macosx_10_13_x86_64.whl


In [7]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [9]:
!pip install lxml
import urllib.request
import zipfile
import lxml.etree


     |████████████████████████████████| 4.5MB 2.2MB/s eta 0:00:01


In [10]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [12]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [13]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [14]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [15]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [16]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [17]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [18]:
len(sentences_ted)

266694

In [19]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [48]:

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer().fit(sentences_strings_ted)
bag_of_words = vec.transform(sentences_strings_ted)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)


In [117]:

# print(words_freq[:400])
counts_ted_top1000 = []
words_top_ted = []
for i in range (0, 1000):
    counts_ted_top1000.append(words_freq[i][1])
    words_top_ted.append(words_freq[i][0])
print(counts_ted_top1000)
# np.array(counts_ted_top1000).astype(np.float)

[207748, 149305, 125169, 114818, 95146, 78069, 74737, 70923, 67629, 63251, 49241, 37014, 33102, 30806, 29713, 27995, 27344, 26732, 26519, 25961, 24706, 23377, 21246, 21041, 20201, 19481, 19216, 19021, 18626, 17928, 17908, 17551, 17011, 16723, 16273, 15868, 15452, 14387, 14061, 13986, 13916, 13896, 13882, 13863, 13278, 12879, 12363, 12298, 12163, 11935, 11595, 11576, 11506, 11445, 11366, 10902, 10900, 10463, 10446, 10180, 10179, 10091, 10029, 10021, 9911, 9872, 9675, 9376, 9308, 9239, 8906, 8841, 8823, 8629, 8262, 8256, 8011, 7858, 7735, 7724, 7617, 7537, 7536, 7281, 7227, 7183, 7168, 7141, 7119, 6937, 6806, 6521, 6424, 6358, 6307, 6292, 6229, 6207, 5972, 5907, 5695, 5684, 5571, 5552, 5501, 5456, 5379, 5374, 5312, 5275, 5220, 5197, 5175, 5153, 5133, 4935, 4929, 4919, 4660, 4546, 4544, 4511, 4485, 4454, 4399, 4299, 4143, 4132, 4130, 4120, 4079, 4034, 3971, 3968, 3798, 3791, 3787, 3786, 3707, 3611, 3494, 3449, 3364, 3346, 3301, 3245, 3217, 3201, 3188, 3171, 3080, 3032, 3031, 2999, 2983, 2

Plot distribution of top-1000 words

In [51]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

/Users/sevenw/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


### Part 3: Train Word2Vec

In [55]:
!pip install gensim
from gensim.models import Word2Vec

     |████████████████████████████████| 24.7MB 1.1MB/s eta 0:00:01    |██                              | 1.5MB 2.3MB/s eta 0:00:11     |████████████████████▎           | 15.6MB 2.3MB/s eta 0:00:04
     |████████████████████████████████| 71kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 39.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 27.6MB/s eta 0:00:01
     |████████████████████████████████| 5.9MB 27.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 552kB 22.1MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=1083394fef8077653bcb2105b720d959d32c9a294e7cd5db0126779b0b949179
  Stored in directory: /Users/sevenw/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [115]:
model_ted = Word2Vec(min_count=10)

In [116]:
model_ted.save("word2vec.model")
model_ted.build_vocab(sentences_ted)
model_ted.train(sentences_ted, total_examples=model_ted.corpus_count, epochs=5)

(15595048, 22334160)

In [112]:
print(len(model_ted.wv.vocab))

14427


In [81]:
x = model_ted.wv.most_similar("man")
y = model_ted.wv.most_similar("computer")
z = model_ted.wv.most_similar("he")
print(x)
print(y)
print(z)

[('woman', 0.8411097526550293), ('guy', 0.8142598271369934), ('lady', 0.7707378268241882), ('girl', 0.7259840965270996), ('boy', 0.7255516052246094), ('gentleman', 0.7173513174057007), ('soldier', 0.7131596207618713), ('kid', 0.6873033046722412), ('david', 0.6802223920822144), ('poet', 0.6797789335250854)]
[('machine', 0.7497755885124207), ('robot', 0.7125699520111084), ('software', 0.6970232725143433), ('device', 0.6758543252944946), ('chip', 0.6681709289550781), ('program', 0.6668930649757385), ('video', 0.6422716379165649), ('camera', 0.6342703104019165), ('mechanical', 0.6210808753967285), ('visualization', 0.6181246042251587)]
[('einstein', 0.6649230718612671), ('she', 0.6585855484008789), ('himself', 0.6249934434890747), ('darwin', 0.5910896062850952), ('david', 0.5808199048042297), ('tony', 0.5773943066596985), ('chris', 0.5747851729393005), ('john', 0.5722081661224365), ('nobody', 0.570104718208313), ('him', 0.568867564201355)]


0.093566
0.09356601


### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [88]:
model_ted.wv.most_similar("man")

[('woman', 0.8411097526550293),
 ('guy', 0.8142598271369934),
 ('lady', 0.7707378268241882),
 ('girl', 0.7259840965270996),
 ('boy', 0.7255516052246094),
 ('gentleman', 0.7173513174057007),
 ('soldier', 0.7131596207618713),
 ('kid', 0.6873033046722412),
 ('david', 0.6802223920822144),
 ('poet', 0.6797789335250854)]

In [89]:
model_ted.wv.most_similar("computer")

[('machine', 0.7497755885124207),
 ('robot', 0.7125699520111084),
 ('software', 0.6970232725143433),
 ('device', 0.6758543252944946),
 ('chip', 0.6681709289550781),
 ('program', 0.6668930649757385),
 ('video', 0.6422716379165649),
 ('camera', 0.6342703104019165),
 ('mechanical', 0.6210808753967285),
 ('visualization', 0.6181246042251587)]

In [90]:
fromModel = model_ted.wv.similarity("man", "computer")
print(fromModel)
fromSta1 = model_ted.wv['man']
fromSta2 = model_ted.wv['computer']
similarity = np.dot(fromSta1, fromSta2) / (np.linalg.norm(fromSta1) * np.linalg.norm(fromSta2))
print(similarity)

0.093566
0.09356601


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [118]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

/Users/sevenw/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [119]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [122]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [123]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [148]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [149]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])


sent_wiki = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sent_wiki.append(tokens)


In [150]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
# sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
# print(len(sentences_wiki))

4267112


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [153]:
print(sent_wiki[:20])
print(sentences_ted[:20])

[['valkyria', 'chronicles', 'iii'], ['senj', 'no', 'valkyria', 'unrecorded', 'chronicles', 'japanese', 'lit'], ['valkyria', 'of', 'the', 'battlefield', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'is', 'a', 'tactical', 'role', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media'], ['vision', 'for', 'the', 'playstation', 'portable'], ['released', 'in', 'january', 'in', 'japan', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series'], ['employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', 'time', 'gameplay', 'as', 'its', 'predecessors', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', 'nameless', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are', 'pitted', 'against', 'the', 'imperial', 'unit', 'calamaty', 'raven'], ['th

In [154]:
model_wiki = Word2Vec(min_count=10)
model_wiki.build_vocab(sent_wiki)
model_wiki.train(sent_wiki, total_examples=model_wiki.corpus_count, epochs=1)
# model_ted.build_vocab(sentences_ted)
# model_ted.train(sentences_ted, total_examples=model_ted.corpus_count, epochs=5)

(63355077, 83270301)

In [167]:
vec = CountVectorizer().fit(sentences_wiki)
bag_of_words = vec.transform(sentences_wiki)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

In [168]:
counts_wiki_top1000 = []
words_top_wiki = []
for i in range (0, 1000):
    counts_wiki_top1000.append(words_freq[i][1])
    words_top_wiki.append(words_freq[i][0])

In [162]:
print(len(model_wiki.wv.vocab))
print(model_wiki.wv.most_similar("man"))
print(model_wiki.wv.most_similar("computer"))

110582
[('woman', 0.7593563199043274), ('boy', 0.6586384177207947), ('girl', 0.6461068987846375), ('soldier', 0.6314491033554077), ('person', 0.6275582313537598), ('dog', 0.6091442108154297), ('mask', 0.6027307510375977), ('thief', 0.59775310754776), ('lad', 0.5939432382583618), ('joker', 0.5938776135444641)]
[('software', 0.8098450899124146), ('hardware', 0.7736421823501587), ('computers', 0.763701319694519), ('simulation', 0.7520653009414673), ('interface', 0.7482790946960449), ('cgi', 0.7308537364006042), ('technology', 0.7258260846138), ('user', 0.719807505607605), ('controller', 0.7197705507278442), ('computing', 0.7031029462814331)]


#### t-SNE visualization

In [169]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/Users/sevenw/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [170]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)